In [ ]:
import pandas as pd
import matplotlib as plt

In [ ]:
d_parser = lambda x:pd.to_datetime(x, errors='coerce')

In [ ]:
df = pd.read_csv('breach_report.csv', encoding= 'latin1', 
                 dtype={'Type of Breach':'string'},
                 parse_dates=['Breach Submission Date'], date_parser=d_parser)

In [ ]:
df.head(5)

,Name of Covered Entity,State,Covered Entity Type,Individuals Affected,Breach Submission Date,Type of Breach,Location of Breached Information,Business Associate Present,Web Description
0,Brooke Army Medical Center,TX,Healthcare Provider,1000.0,2009-10-21,Theft,Paper/Films,No,A binder containing the protected health infor...
1,"Mid America Kidney Stone Association, LLC",MO,Healthcare Provider,1000.0,2009-10-28,Theft,Network Server,No,Five desktop computers containing unencrypted ...
2,Alaska Department of Health and Social Services,AK,Healthcare Provider,501.0,2009-10-30,Theft,"Other, Other Portable Electronic Device",No,\N
3,Health Services for Children with Special Need...,DC,Health Plan,3800.0,2009-11-17,Loss,Laptop,No,A laptop was lost by an employee while in tran...
4,"Mark D. Lurie, MD",CA,Healthcare Provider,5166.0,2009-11-20,Theft,Desktop Computer,No,A shared Computer that was used for backup was...


In [ ]:
df.dtypes

Name of Covered Entity                      object
State                                       object
Covered Entity Type                         object
Individuals Affected                       float64
Breach Submission Date              datetime64[ns]
Type of Breach                              string
Location of Breached Information            object
Business Associate Present                  object
Web Description                             object
dtype: object

In [ ]:
df.shape

(1700, 9)

In [ ]:
df1 = df.loc[(df['Breach Submission Date'] <= '2013-09-30')]

In [ ]:
#df1.groupby(df1['Breach Submission Date'])[['Breach Submission Date']].sum()

In [ ]:
df1.isna().sum(axis = 0)

Name of Covered Entity               0
State                                6
Covered Entity Type                 23
Individuals Affected                14
Breach Submission Date               0
Type of Breach                      10
Location of Breached Information     7
Business Associate Present           0
Web Description                     11
dtype: int64

In [ ]:
round(df1.isna().sum() * 100/len(df),2)

Name of Covered Entity              0.00
State                               0.35
Covered Entity Type                 1.35
Individuals Affected                0.82
Breach Submission Date              0.00
Type of Breach                      0.59
Location of Breached Information    0.41
Business Associate Present          0.00
Web Description                     0.65
dtype: float64

In [ ]:
df1 = df1.dropna(subset = ['State','Covered Entity Type','Individuals Affected','Type of Breach','Location of Breached Information', 'Web Description'])

In [ ]:
round(df1.isna().sum() * 100/len(df1),2)

Name of Covered Entity              0.0
State                               0.0
Covered Entity Type                 0.0
Individuals Affected                0.0
Breach Submission Date              0.0
Type of Breach                      0.0
Location of Breached Information    0.0
Business Associate Present          0.0
Web Description                     0.0
dtype: float64

In [ ]:
df1.groupby(['Type of Breach']).sum()

,Individuals Affected
Type of Breach,
Hacking/IT Incident,1231391.0
"Hacking/IT Incident, Other",3200.0
"Hacking/IT Incident, Other, Unauthorized Access/Disclosure",4354.0
"Hacking/IT Incident, Theft",27800.0
"Hacking/IT Incident, Theft, Unauthorized Access/Disclosure",13800.0
"Hacking/IT Incident, Unauthorized Access/Disclosure",62487.0
Improper Disposal,620927.0
"Improper Disposal, Loss",3430.0
"Improper Disposal, Loss, Theft",1950.0


In [ ]:
for x in df1['Type of Breach']:
  if 'Hacking/IT Incident' in x:
    df1.loc[df1['Type of Breach'] == x,'Breach Type'] = 'Hacking/IT Incident'
    continue
  if 'Improper Disposal' in x:
    df1.loc[df1['Type of Breach'] == x,'Breach Type'] = 'Improper Disposal'
    continue
  if 'Loss' in x:
    df1.loc[df1['Type of Breach'] == x,'Breach Type'] = 'Loss'
    continue
  if 'Unauthorized Access/Disclosure' in x:
    df1.loc[df1['Type of Breach'] == x,'Breach Type'] = 'Unauthorized Access/Disclosure'
    continue
  if 'Theft' in x:
    df1.loc[df1['Type of Breach'] == x,'Breach Type'] = 'Theft'
    continue
  if 'Unknown' in x or 'Other' in x:
    df1.loc[df1['Type of Breach'] == x,'Breach Type'] = 'Other/Unkown'
    continue

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
df1.head()

,Name of Covered Entity,State,Covered Entity Type,Individuals Affected,Breach Submission Date,Type of Breach,Location of Breached Information,Business Associate Present,Web Description,Breach Type
0,Brooke Army Medical Center,TX,Healthcare Provider,1000.0,2009-10-21,Theft,Paper/Films,No,A binder containing the protected health infor...,Theft
1,"Mid America Kidney Stone Association, LLC",MO,Healthcare Provider,1000.0,2009-10-28,Theft,Network Server,No,Five desktop computers containing unencrypted ...,Theft
2,Alaska Department of Health and Social Services,AK,Healthcare Provider,501.0,2009-10-30,Theft,"Other, Other Portable Electronic Device",No,\N,Theft
3,Health Services for Children with Special Need...,DC,Health Plan,3800.0,2009-11-17,Loss,Laptop,No,A laptop was lost by an employee while in tran...,Loss
4,"Mark D. Lurie, MD",CA,Healthcare Provider,5166.0,2009-11-20,Theft,Desktop Computer,No,A shared Computer that was used for backup was...,Theft


In [ ]:
df1.groupby(['Breach Type']).sum()

,Individuals Affected
Breach Type,
Hacking/IT Incident,1343032.0
Improper Disposal,644108.0
Loss,7101794.0
Other/Unkown,2781311.0
Theft,13852804.0
Unauthorized Access/Disclosure,936979.0
